<a href="https://colab.research.google.com/github/LinaMariaCastro/curso-ia-para-economia/blob/main/clases/3_Analisis_y_visualizacion_datos/4_Manipulacion_y_Transformacion_de_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wbdata

# **Inteligencia Artificial con Aplicaciones en Economía I**

- 👩‍🏫 **Profesora:** [Lina María Castro](https://www.linkedin.com/in/lina-maria-castro)  
- 📧 **Email:** [lmcastroco@gmail.com](mailto:lmcastroco@gmail.com)  
- 🎓 **Universidad:** Universidad Externado de Colombia - Facultad de Economía

# 🪄**Manipulación y transformación de datos**

👉 Ahora que los datos están limpios, toca organizarlos para analizarlos.

✅ Resultado: un dataset estructurado para responder preguntas económicas.

**Objetivos de Aprendizaje:**

Al finalizar este notebook, serás capaz de:

1. **Organizar y filtrar datos** para aislar la información más relevante para una pregunta de investigación económica.

2. **Agregar y resumir datos** para calcular estadísticas descriptivas a nivel de grupo (e.g., por región, por nivel de ingreso).

3. **Remodelar y transformar tablas de datos** para pasar de formatos anchos a largos y viceversa, una habilidad crucial para preparar datos para modelos econométricos y visualizaciones.

**Introducción**

Imagina que eres un economista recién contratado en el Banco Mundial. Tu primer encargo es analizar la relación entre el Gasto Público en Educación (% del PIB) y el Crecimiento del PIB per cápita para diferentes regiones del mundo durante la última década.

Recibes un enorme archivo csv con miles de filas y columnas, mezclando países, años e indicadores de todo tipo. No puedes simplemente introducir eso en un modelo. Primero, necesitas esculpir y analizar tus datos. Necesitas:

- Ordenar los países para ver los que más y menos invierten.

- Filtrar para quedarte solo con los años y los indicadores relevantes.

- Agrupar por región para calcular el gasto promedio y ver si África Subsahariana invierte, en promedio, más o menos que América Latina.

- Pivotar la tabla para que cada fila sea un país y cada columna un año, permitiéndote ver la evolución de un vistazo.

Estas no son solo "funciones de pandas"; son las herramientas diarias de un economista.

## Importar librerías

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Mejorar visualización de los dataframes

In [3]:
# Que muestre todas las columnas
pd.options.display.max_columns = None
# En los dataframes, mostrar los float con dos decimales
pd.options.display.float_format = '{:,.2f}'.format

## Cargar el dataset

Como ejemplo, usaremos un **dataset real del Banco Mundial** que contiene indicadores de desarrollo. Esto nos permitirá realizar un análisis que un economista haría en su día a día.

Utilizaremos la API del Banco Mundial para obtener los datos.

In [4]:
import wbdata

In [5]:
# Listar bases de datos disponibles en el Banco Mundial
# Nos interesa World Development Indicators
wbdata.get_sources()

  id  name
----  --------------------------------------------------------------------
   1  Doing Business
   2  World Development Indicators
   3  Worldwide Governance Indicators
   5  Subnational Malnutrition Database
   6  International Debt Statistics
  11  Africa Development Indicators
  12  Education Statistics
  13  Enterprise Surveys
  14  Gender Statistics
  15  Global Economic Monitor
  16  Health Nutrition and Population Statistics
  18  IDA Results Measurement System
  19  Millennium Development Goals
  20  Quarterly Public Sector Debt
  22  Quarterly External Debt Statistics SDDS
  23  Quarterly External Debt Statistics GDDS
  25  Jobs
  27  Global Economic Prospects
  28  Global Findex database
  29  The Atlas of Social Protection: Indicators of Resilience and Equity
  30  Exporter Dynamics Database – Indicators at Country-Year Level
  31  Country Policy and Institutional Assessment
  32  Global Financial Development
  33  G20 Financial Inclusion Indicators
  34  Global P

In [6]:
# Listar indicadores disponibles en World Development Indicators
wbdata.get_indicators(source=2)

id                          name
--------------------------  ---------------------------------------------------------------------------------------------------------------------------------------------
AG.CON.FERT.PT.ZS           Fertilizer consumption (% of fertilizer production)
AG.CON.FERT.ZS              Fertilizer consumption (kilograms per hectare of arable land)
AG.LND.AGRI.K2              Agricultural land (sq. km)
AG.LND.AGRI.ZS              Agricultural land (% of land area)
AG.LND.ARBL.HA              Arable land (hectares)
AG.LND.ARBL.HA.PC           Arable land (hectares per person)
AG.LND.ARBL.ZS              Arable land (% of land area)
AG.LND.CREL.HA              Land under cereal production (hectares)
AG.LND.CROP.ZS              Permanent cropland (% of land area)
AG.LND.EL5M.RU.K2           Rural land area where elevation is below 5 meters (sq. km)
AG.LND.EL5M.RU.ZS           Rural land area where elevation is below 5 meters (% of total land area)
AG.LND.EL5M.UR.K2  

In [7]:
# Buscar los indicadores de interés en World Development Indicators
wbdata.get_indicators(query="GDP per capita", source=2)

id                 name
-----------------  -------------------------------------------------------------------
NY.GDP.PCAP.CD     GDP per capita (current US$)
NY.GDP.PCAP.CN     GDP per capita (current LCU)
NY.GDP.PCAP.KD     GDP per capita (constant 2015 US$)
NY.GDP.PCAP.KD.ZG  GDP per capita growth (annual %)
NY.GDP.PCAP.KN     GDP per capita (constant LCU)
NY.GDP.PCAP.PP.CD  GDP per capita, PPP (current international $)
NY.GDP.PCAP.PP.KD  GDP per capita, PPP (constant 2021 international $)
SE.XPD.PRIM.PC.ZS  Government expenditure per student, primary (% of GDP per capita)
SE.XPD.SECO.PC.ZS  Government expenditure per student, secondary (% of GDP per capita)
SE.XPD.TERT.PC.ZS  Government expenditure per student, tertiary (% of GDP per capita)

In [9]:
indicadores = {"NY.GDP.PCAP.PP.CD": "PIB per cápita (PPA, USD internacionales)",
               "SH.XPD.CHEX.PP.CD": "Gasto en salud per cápita (PPA, USD internacionales)",
               "SP.DYN.LE00.IN": "Esperanza de vida (años)",
               "SL.UEM.TOTL.NE.ZS": "Tasa de desempleo (%)"}

# Descargar datos para países de América del Sur de 2010 a 2022
df = wbdata.get_dataframe(indicadores, country=["COL", "PER", "BRA", "CHL", "ARG", "BOL", "ECU", "PRY", "URY"], date=("2010", "2022"))
df = df.reset_index()
df = df.rename(columns={'country': 'País', 'date': 'Año'})

## Exploración inicial del dataset

In [10]:
df

,País,Año,"PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)",Esperanza de vida (años),Tasa de desempleo (%)
0,Argentina,2022,"29,597.69","2,664.35",75.81,6.80
1,Argentina,2021,"26,300.27","2,418.80",73.95,8.74
2,Argentina,2020,"22,393.35","2,182.31",75.88,11.46
3,Argentina,2019,"23,516.83","2,334.76",76.85,9.84
4,Argentina,2018,"24,410.39","2,388.69",76.77,9.22
...,...,...,...,...,...,...
112,Uruguay,2014,"21,867.81","1,785.41",77.19,6.55
113,Uruguay,2013,"20,661.38","1,709.51",76.95,6.44
114,Uruguay,2012,"19,495.42","1,615.23",76.69,6.45
115,Uruguay,2011,"19,135.36","1,574.28",76.61,6.31


In [11]:
df.shape

(117, 6)

In [12]:
df.info()

<class 'wbdata.client.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 6 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   País                                                  117 non-null    object 
 1   Año                                                   117 non-null    object 
 2   PIB per cápita (PPA, USD internacionales)             117 non-null    float64
 3   Gasto en salud per cápita (PPA, USD internacionales)  117 non-null    float64
 4   Esperanza de vida (años)                              117 non-null    float64
 5   Tasa de desempleo (%)                                 113 non-null    float64
dtypes: float64(4), object(2)
memory usage: 5.6+ KB


In [13]:
df["País"].value_counts(dropna=False) # este me realiza un conteo por país

,count
País,
Argentina,13
Bolivia,13
Brazil,13
Chile,13
Colombia,13
Ecuador,13
Peru,13
Paraguay,13
Uruguay,13


Tenemos una base de datos con información de 9 países de América del Sur para 13 años (2010-2022), lo que nos da un total de 117 filas. Se trajo información de 4 indicadores. La información se presenta en una estructura de panel de datos.

## Realizar filtros

En ocasiones, un economista quiere seleccionar subconjuntos de datos para analizar. Por ejemplo, podemos estar interesados en conocer cómo se comportaron los indicadores en los distintos países en el año 2022, por lo que vamos a crear un dataframe solo con la información de 2022.

In [ ]:
df_2022 = df[df['Año'] == "2022"]
df_2022

In [ ]:
# Reiniciamos el índice
df_2022.reset_index(drop=True, inplace=True)
df_2022

### Filtrar por el índice

Queremos ver la información correspondiente al índice 3

In [ ]:
df_2022.iloc[3]

### Filtrar por una columna

Queremos ver la información de Colombia

In [ ]:
df_2022[df_2022['País']=="Colombia"]

**Ejercico**

Muéstrame todos los países que sean diferentes a Colombia

**Ejercicio**

¿Qué países tienen una tasa de desempleo menor estrictamente a 5%?

### Filtrar por varias columnas

¿Qué países tienen una tasa de desempleo mayor o igual a 6% y una esperanza de vida mayor o igual a 75 años?

In [ ]:
df_2022[(df_2022['Tasa de desempleo (%)']>=6) & (df_2022['Esperanza de vida (años)']>=75)]

¿Qué países tienen un PIB per cápita PPA mayor a USD 25.000 o un gasto en salud per cápita PPA mayor a USD 1.000?

In [ ]:
df_2022[(df_2022['PIB per cápita (PPA, USD internacionales)']>25000) | (df_2022['Gasto en salud per cápita (PPA, USD internacionales)']>1000)]

### Filtrar por el mínimo o máximo de una columna

¿Cuál país tiene el PIB per cápita más alto?

In [ ]:
df_2022[df_2022['PIB per cápita (PPA, USD internacionales)'] == df_2022['PIB per cápita (PPA, USD internacionales)'].max()]

**Ejercicio**

¿Cuál país tiene la esperanza de vida más baja?

**Ejercicio**

¿Qué países tienen un gasto en salud per cápita PPA por encima del promedio?

### Filtrar por una lista de valores

Muéstrame los datos para Brasil y Ecuador

In [ ]:
df_2022[df_2022['País'].isin(['Brazil','Ecuador'])]

### Filtrar los strings que contengan ciertos caracteres

Muéstrame los países que contengan "P"

In [ ]:
df_2022[df_2022['País'].str.contains("P", na=False)]

Muéstrame los países que contengan "Pe" o "Br" o "Ch"

In [ ]:
df_2022[df_2022['País'].str.contains(r'Pe|Br|Ch', na=False)]

**Ejercicio**

Muestra los países que contengan "guay"

### Filtrar por negación (todo excepto ...)

Muéstrame todos los países excepto Brasil

In [ ]:
df_2022[~(df_2022['País']=='Brazil')]

**Ejercicio**

Muéstrame todos los países excepto los que terminan en "guay". Filtra utilizando negación.

**Ejercicio**

Muestra los países donde su PIB per cápita no sea mayor a USD 25.000 PPA y su tasa de desempleo no sea menor a 5%. Filtra utilizando negación.

## Ordenar el dataframe

Para un economista, ordenar es sinónimo de ranking. ¿Qué país tiene el mayor PIB? ¿Cuál tiene la menor tasa de inflación? sort_values() nos permite responder a estas preguntas.

### Ordenar según el valor de una columna

Vamos a ordenar los países de menor a mayor gasto en salud per cápita

In [ ]:
df_2022.sort_values(by=['Gasto en salud per cápita (PPA, USD internacionales)'], ascending = True)

**Ejercicio**

Ordena los países de mayor a menor PIB per cápita PPA

**Ejercicio**

 Encuentra los 5 países con la menor tasa de desempleo en 2022.

### Ordenar según el valor de varias columnas

In [ ]:
# Datos económicos sintéticos de varios países
datos = {
    'País': ['Colombia', 'Chile', 'México', 'Argentina', 'Perú', 'Panamá'],
    'Región': ['Andina', 'Andina', 'Norteamérica', 'Sur', 'Andina', 'Centroamérica'],
    'Crecimiento_PIB_%': [1.5, 2.1, 2.5, -1.1, 2.8, 4.1],
    'Tasa_Desempleo_%': [10.5, 8.8, 3.5, 11.2, 7.5, 8.1]
}

# Creamos el DataFrame
df_ejemplo = pd.DataFrame(datos)

df_ejemplo

In [ ]:
df_ordenado = df_ejemplo.sort_values(by=['Región', 'Crecimiento_PIB_%'], ascending=[True, False])
df_ordenado

## Groupby

Esta es una de las herramientas más utilizadas, ya que permite agrupar por categorías y sacar mediciones para cada grupo como la media, la suma, la desviación estándar, el conteo de registros, etc.

Para ello, se emplea la estrategia de Dividir-Aplicar-Combinar.

- **Dividir (Split):** Se separa el DataFrame en grupos basados en una o más variables categóricas (ejemplo: País, Año, etc).

- **Aplicar (Apply):** A cada grupo se le aplica una función (ejemplo: mean, sum, std).

- **Combinar (Combine):** Se juntan los resultados en un nuevo DataFrame.

Es como si el FMI quisiera calcular la tasa de inflación promedio para cada continente. Primero, divide el dataset global por continente. Luego, aplica la función de promedio a la columna de inflación para cada grupo. Finalmente, combina los resultados en una tabla resumen.

In [ ]:
# Dataframe original
df

¿Cuál fue la esperanza de vida promedio de cada país entre 2010 y 2022?

In [ ]:
df.groupby('País')['Esperanza de vida (años)'].mean().reset_index()

Ordenemos de mayor a menor...

In [ ]:
df.groupby('País')['Esperanza de vida (años)'].mean().reset_index().sort_values(by='Esperanza de vida (años)', ascending=False)

¿Cuál fue la tasa de desempleo más baja que registró cada país entre 2010 y 2022?

In [ ]:
df.groupby('País')['Tasa de desempleo (%)'].min().reset_index()

¿Cuánto sumó el gasto en salud per cápita de estos países en cada uno de los años de estudio (2010-2022)?

In [ ]:
df.groupby('Año')['Gasto en salud per cápita (PPA, USD internacionales)'].sum().reset_index()

Para cada país de América del Sur, ¿cuál fue el promedio de PIB per cápita PPA y de gasto en salud per cápita PPA entre 2010 y 2022?

In [ ]:
df.groupby("País")[["PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)"]].mean().reset_index()

Para cada país de América del Sur, ¿cuál fue el promedio de gasto en salud per cápita PPA y cuál fue la máxima esperanza de vida entre 2010 y 2022?

In [ ]:
df.groupby("País", as_index=False).agg(
    gasto_promedio_salud = ("Gasto en salud per cápita (PPA, USD internacionales)", "mean"),
    max_esperanza_de_vida = ("Esperanza de vida (años)", "max")
)

Para cada país de América del Sur, calcule el mínimo, el máximo y el promedio del gasto en salud per cápita PPA y de la esperanza de vida entre 2010 y 2022

In [ ]:
df.groupby("País")[["Gasto en salud per cápita (PPA, USD internacionales)","Esperanza de vida (años)"]].agg(["min", "mean", "max"]).reset_index()

**Ejercicio**

Para cada país de América del Sur, calcule la mediana y la desviación estándar del PIB per cápita PPA entre 2010 y 2022

## Tablas pivote o dinámicas

Una tabla pivote es una forma avanzada de groupby que nos **permite remodelar el DataFrame**, poniendo una variable categórica como índice, otra como columnas y una variable numérica como los valores. **Es similar a las tablas dinámicas en Excel.**

Por ejemplo, creemos una tabla para un informe donde las filas son los países, las columnas son los años (2010, 2011, 2012...), y los valores en las celdas son la tasa de desempleo. Esto facilita enormemente la comparación de la evolución del desempleo entre países.

In [ ]:
df_desempleo = df.pivot_table(index='País', columns='Año', values='Tasa de desempleo (%)').reset_index()
df_desempleo

Este formato **ancho (wide)** es muy intuitivo. Podemos comparar fácilmente el desempleo en 2022 de los diferentes países de América del Sur y ver que el de Colombia es el más alto.

También podemos seguir la fila de un país para ver su tendencia, como por ejemplo, ver el comportamiento que ha tenido la tasa de desempleo de Colombia entre 2010 y 2022 y ver que durante la pandemia llegó a un máximo de 16%.


A continuación, vamos a crear una tabla que presenta, para cada país, la evolución 2010-2022 del PIB per cápita PPA y de la tasa de desempleo.

In [ ]:
df1 = pd.pivot_table(df, index = 'País', columns = 'Año',
                     values = ['PIB per cápita (PPA, USD internacionales)',
                               'Tasa de desempleo (%)']).reset_index()
df1

En la siguiente tabla, vamos a mostrar, para cada país, el valor promedio del PIB per cápita PPA y de la tasa de desempleo entre 2010 y 2022.

In [ ]:
df2 = pd.pivot_table(df, index = 'País', values = ['PIB per cápita (PPA, USD internacionales)',
                                                   'Tasa de desempleo (%)'],
                     aggfunc='mean').reset_index()
df2

A continuación se muestra, para cada país, el valor promedio y la desviación estándar del PIB per cápita PPA y el máximo y mínimo de la tasa de desempleo en el período 2010-2022.

In [ ]:
df3 = pd.pivot_table(df, index = 'País', values = ['PIB per cápita (PPA, USD internacionales)',
                                                   'Tasa de desempleo (%)'],
                     aggfunc={'PIB per cápita (PPA, USD internacionales)':['mean','std'],
                              'Tasa de desempleo (%)':['min', 'max']}).reset_index()
df3

## Transponer un dataframe

Dado que un dataframe es una matriz, lo podemos transponer.

In [ ]:
# Dataframe inicial
df_desempleo

In [ ]:
# Dataframe transpuesto
df_desempleo_2 = df_desempleo.set_index('País').transpose().reset_index()
df_desempleo_2

## Reshape del dataframe

Transformando Formatos: de long a wide y de wide a long.


**Formato Largo (Long):** Cada observación está en su propia fila. (Ej: Tenemos columnas de País, Año, Indicador, Valor, por lo que cada fila es una observación). Es el formato ideal para almacenar datos.

**Formato Ancho (Wide):** Cada fila representa una unidad de observación (como un país) y las variables se extienden a lo largo de las columnas (Ej: Country, GDP_2010, GDP_2011, GDP_2012). Es ideal para analizar los datos.

### Pasar de wide a long

In [ ]:
# Dataframe original
df_desempleo

In [ ]:
# Convertir a formato long
df_largo_desempleo = pd.melt(
    df_desempleo,
    id_vars=['País'], # Columnas que se mantienen fijas
    var_name='Año', # Nombre de la nueva columna de variables
    value_name='Desempleo (%)' # Nombre de la nueva columna de valores
)
df_largo_desempleo

El dataframe del Banco Mundial, que tiene forma de panel de datos y es wide, también lo podemos convertir a long.

In [ ]:
df_largo = pd.melt(
    df,
    id_vars=['País', 'Año'], # Columnas que se mantienen fijas
    var_name='Indicador', # Nombre de la nueva columna de variables
    value_name='Valor' # Nombre de la nueva columna de valores
)
df_largo

### Pasar de long a wide

Para pasar de formato long a wide se utilizan las tablas pivote que vimos previamente.

## Ejercicio

Utiliza las tablas pivote para obtener, a partir de *df_largo*, un dataframe en formato wide de la esperanza de vida (en cada fila hay un país y en cada columna van los años de 2010 a 2022). Luego transforma el dataframe para que los años queden en las filas y los países en las columnas.